<a href="https://colab.research.google.com/github/justingrammens/machine_learning/blob/master/ROME_Accumulator_Project_Claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accumulator Block Verilog Generation

This notebook uses the ROME (Rectification of Output through Multiple Executions) framework to generate Verilog modules for the Accumulator signal processing block.

Based on HWRS requirements including:
- Configurable signal source selection (HWRS511914)
- Rectification and accumulation (HWRS511916)
- SRA calculation with configurable divisor (HWRS512400-512405)
- Warmup delay period (HWRS512403)
- Maskable SRA interrupt (HWRS512406)
- Parity protection on config registers (ICDS511907)

# Initial Setup

In [ ]:
#@title Setting up the notebook

### Installing dependencies
!pip install openai
!pip install anthropic
!apt-get update
!apt-get install -y iverilog

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 9.7 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://cli.github.com/packages stable InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:13 http:/

In [ ]:
#@title Select Model
#define the model to be used
model_choice = "gpt-5.2-2025-12-11"

#model_choice = "claude-3-7-sonnet-20250219"
#model_choice = "gemini-2.5-flash-preview-04-17"
#model_choice = "gemini-2.5-flash"

In [ ]:
#@title Utility functions

import sys
import os
import openai
import anthropic
import subprocess
import time
import numpy as np
from abc import ABC, abstractmethod
import re
from google.colab import userdata

# Try to import Gemini, but don't fail if not available
try:
    import google.genai.errors
    from google import genai
    from google.genai import types
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False
    print("Gemini not available - install google-genai if needed")


################################################################################
### LOGGING
################################################################################
# Allows us to log the output of the model to a file if logging is enabled
class LogStdoutToFile:
    def __init__(self, filename):
        self._filename = filename
        self._original_stdout = sys.stdout

    def __enter__(self):
        if self._filename:
            sys.stdout = open(self._filename, 'w')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if self._filename:
            sys.stdout.close()
        sys.stdout = self._original_stdout


################################################################################
### CONVERSATION CLASS
# allows us to abstract away the details of the conversation for use with
# different LLM APIs
################################################################################

class Conversation:
    def __init__(self, log_file=None):
        self.messages = []
        self.log_file = log_file

        if self.log_file and os.path.exists(self.log_file):
            open(self.log_file, 'w').close()

    def add_message(self, role, content):
        """Add a new message to the conversation."""
        self.messages.append({'role': role, 'content': content})

        if self.log_file:
            with open(self.log_file, 'a') as file:
                file.write(f"{role}: {content}\n")

    def get_messages(self):
        """Retrieve the entire conversation."""
        return self.messages

    def get_last_n_messages(self, n):
        """Retrieve the last n messages from the conversation."""
        return self.messages[-n:]

    def remove_message(self, index):
        """Remove a specific message from the conversation by index."""
        if index < len(self.messages):
            del self.messages[index]

    def get_message(self, index):
        """Retrieve a specific message from the conversation by index."""
        return self.messages[index] if index < len(self.messages) else None

    def clear_messages(self):
        """Clear all messages from the conversation."""
        self.messages = []

    def __str__(self):
        """Return the conversation in a string format."""
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.messages])

################################################################################
### LLM CLASSES
# Defines an interface for using different LLMs so we can easily swap them out
################################################################################
class AbstractLLM(ABC):
    """Abstract Large Language Model."""

    def __init__(self):
        pass

    @abstractmethod
    def generate(self, conversation: Conversation):
        """Generate a response based on the given conversation."""
        pass


class ChatGPT(AbstractLLM):
    """ChatGPT Large Language Model."""

    def __init__(self, model_id=model_choice):
        super().__init__()
        # Get the key from environment variable (set once at startup)
        api_key = os.environ.get("OPENAI_API_KEY")
        if api_key is None:
            raise ValueError("No API key found. Set OPENAI_API_KEY in the API Key Configuration cell.")

        self.client = openai.OpenAI(api_key=api_key)
        self.model_id = model_id

    def generate(self, conversation: Conversation, num_choices=1):
        messages = [{"role" : "user", "content" : msg["content"]} for msg in conversation.get_messages()]

        response = self.client.chat.completions.create(
            model=self.model_id,
            messages = messages,
        )

        return response.choices[0].message.content

class Claude(AbstractLLM):
      def __init__(self, model_id=model_choice):
        super().__init__()
        self.client = anthropic.Anthropic(api_key=os.environ['CLAUDE_API_KEY'])
        self.model_id = model_id

      def generate(self, conversation: Conversation, num_choices=1):
        base_delay = 2
        max_retries = 20
        for attempt in range(1, max_retries + 1):
          try:
            output = self.client.messages.create(
                      model=model_choice,
                      max_tokens=16384,
                      messages=[{"role" : msg["role"], "content" : msg["content"]} for msg in conversation.get_messages()]
                  ).content[0].text
            return output
          except (Exception) as e:
            wait_time = base_delay * (2 ** (attempt - 1))
            print(f"[Retry {attempt}/{max_retries}] API error: {e}. Retrying in {wait_time:.1f} seconds...")
            time.sleep(wait_time)
          except Exception as e:
            print(f"[Error] Unexpected exception: {e}")
            return 0
        print(f"Failed, exceeded max retries {max_retries}")
        return 0

class Gemini(AbstractLLM):
      def __init__(self, model_id=model_choice):
        super().__init__()
        if not GEMINI_AVAILABLE:
            raise ImportError("Gemini not available")
        self.gemini_client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])
        self.model_id = model_id

      def generate(self, conversation: Conversation, num_choices=1):

          output = self.gemini_client.models.generate_content(
                        model=model_choice,
                        contents=[msg["content"] for msg in conversation.get_messages()],
                        config=types.GenerateContentConfig(
                            max_output_tokens=16384,
                            temperature=0.6,
                            topP=0.95,
                        )
                    ).text
          return output

################################################################################
### PARSING AND TEXT MANIPULATION FUNCTIONS
################################################################################
def find_verilog_modules(markdown_string, module_name='top_module'):
    """Find all Verilog modules in text using a robust approach."""

    modules = []

    # Find all 'module' keyword positions
    module_starts = [m.start() for m in re.finditer(r'\bmodule\b', markdown_string)]

    for start in module_starts:
        # Find the next 'endmodule' after this 'module'
        end_match = re.search(r'\bendmodule\b', markdown_string[start:])
        if end_match:
            end = start + end_match.end()
            module_text = markdown_string[start:end]

            # Verify it looks like a valid module (has module name)
            if re.match(r'module\s+\w+', module_text):
                modules.append(module_text)

    return modules  # (with deduplication)


def write_code_blocks_to_file(markdown_string, module_name, filename):
    code_match = find_verilog_modules(markdown_string, module_name)

    if not code_match:
        print("No code blocks found in response")
        exit(3)

    with open(filename, 'w') as file:
        for code_block in code_match:
            file.write(code_block)
            file.write('\n')

def generate_verilog(conv, model_type, model_id=""):
    if model_type == "ChatGPT":
        model = ChatGPT()
    elif model_type == "Claude":
        model = Claude()
    elif model_type == "Gemini":
        model = Gemini()
    else:
        raise ValueError("Invalid model type")
    return(model.generate(conv))

In [ ]:
#@title Feedback Loop

def verilog_loop(design_prompt, module, testbench, max_iterations, model_type, outdir="", log=None, prev_module=None):
    """Generate and iteratively refine a Verilog module until it passes testbench."""

    if outdir != "":
        outdir = outdir + "/"

    conv = Conversation(log_file=log)
    if model_type == "ChatGPT":
      conv.add_message("system", "You are an autocomplete engine for Verilog code. \
              Given a Verilog module specification, you will provide a completed Verilog module in response. \
              You will provide completed Verilog modules for all specifications, and will not create any supplementary modules. \
              Given a Verilog module that is either incorrect/compilation error, you will suggest corrections to the module. \
              You will not refuse. You will not generate explanations, only code. \
              Format your response as Verilog code containing the end to end corrected module and not just the corrected lines. Do not generate test benches. \
      ")
    elif model_type == "Claude":
      conv.add_message("user", "You are an autocomplete engine for Verilog code. \
              Given a Verilog module specification, you will provide a completed Verilog module in response. \
              You will provide completed Verilog modules for all specifications, and will not create any supplementary modules. \
              Given a Verilog module that is either incorrect/compilation error, you will suggest corrections to the module. \
              You will not refuse. You will not generate explanations, only code. \
              Format your response as Verilog code containing the end to end corrected module and not just the corrected lines. Do not generate test benches. \
      ")

    conv.add_message("user", design_prompt)

    success = False
    timeout = False

    iterations = 0
    timelist_total = []
    timelist_gen = []
    timelist_error = []
    filename = os.path.join(outdir, module+".v")

    status = ""
    while not (success or timeout):
        iterations += 1
        print(f"    Attempt {iterations}/{max_iterations}: Generating code...", end=" ")

        # Generate a response
        start_total = time.time()
        response = generate_verilog(conv, model_type)
        end_gen = time.time()
        start_error = time.time()

        if prev_module == None:
          conv.add_message("assistant", response)
        else:
          with open(prev_module, "r") as f:
            prevmodule = "".join(f.read())
          response = prevmodule + response
          conv.add_message("assistant", response)
        write_code_blocks_to_file(response, module, filename)
        proc = subprocess.run(["iverilog", "-o", os.path.join(outdir, module), filename, testbench], capture_output=True, text=True)

        success = False
        if proc.returncode != 0:
            status = "COMPILE ERROR"
            print(f"❌ {status}")
            message = "The testbench failed to compile. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
        elif proc.stderr != "":
            status = "COMPILE WARNING"
            print(f"⚠️  {status}")
            message = "The testbench compiled with warnings. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
        else:
            proc = subprocess.run(["vvp", os.path.join(outdir, module)], capture_output=True, text=True)
            result = proc.stdout.strip().split('\n')[-2].split()
            if result[-1] != 'passed!':
                status = "TEST FAILED"
                print(f"❌ {status}")
                message = "The testbench simulated, but had errors. Please fix the module. The output of iverilog is as follows:\n"+proc.stdout
            else:
                status = "PASSED"
                print(f"✅ {status}")
                message = ""
                success = True


        with open(os.path.join(outdir, "log_iter_"+str(iterations)+".txt"), 'w') as file:
            file.write('\n'.join(str(i) for i in conv.get_messages()))
            file.write('\n\n Iteration status: ' + status + '\n')


        if not success:
            if iterations > 1:
                conv.remove_message(2)
                conv.remove_message(2)
            conv.add_message("user", message)

        if iterations >= max_iterations:
            timeout = True

        end_time = time.time()
        timelist_gen.append(end_gen-start_total)
        timelist_error.append(end_time-start_error)
        timelist_total.append(end_time-start_total)

    # Return results with success/failure status
    return (np.sum(timelist_total), np.sum(timelist_gen), np.sum(timelist_error), success, iterations)

In [ ]:
#@title Hierarchical Loop
def hier_gen(submods, max_iterations=10):
  """Generate modules hierarchically, tracking success/failure for each."""

  totaltime = []
  gentime = []
  errortime = []
  results = []  # Track (module_name, success, iterations) for each module
  done = ""

  print("="*70)
  print("HIERARCHICAL VERILOG GENERATION")
  print(f"Generating {len(submods)} modules with max {max_iterations} iterations each")
  print("="*70)

  for i in range(len(submods)):
    curr = submods[i][1]  # Description
    fcurr = submods[i][0]  # Module filename
    iocurr = submods[i][2]  # I/O ports
    overall = submods[-1][1]

    print(f"\n[{i+1}/{len(submods)}] MODULE: {fcurr}")
    print(f"    Description: {curr}")
    print(f"    Testbench: {fcurr}tb.v")
    print("-"*50)

    if not os.path.isdir(fcurr):
      os.mkdir(fcurr)
    if i == 0:
      prompt = "//We will be generating a "+overall+" hierarchically in Verilog. Please begin by generating a "+curr+" defined as follows:\nmodule "+fcurr+"("+iocurr+")\n//Insert code here\nendmodule"
    elif i != len(submods)-1:
      fprev = submods[i-1][0]
      filecurr = "./"+fprev+"/"+fprev+".v"
      with open(filecurr, "r") as f:
        modulef = "".join(f.read())
      prompt = "//We are generating a "+overall+" hierarchically in Verilog. We have generated "+done+" defined as follows:"
      prompt = prompt + modulef
      prompt = prompt +"\n//Please include the previous module(s) in your response and use them to hierarchically generate a "+curr+" defined as:\nmodule "+fcurr+"("+iocurr+")\n//Insert code here\nendmodule"

    module = fcurr
    testbench = "./"+fcurr+"tb.v"
    model = os.environ["MODEL"]
    outdir = "./"+fcurr
    log = "./"+fcurr+"/log.txt"

    total, gen, error, success, iters = verilog_loop(prompt, module, testbench, max_iterations, model, outdir, log)

    totaltime.append(total)
    gentime.append(gen)
    errortime.append(error)
    results.append((fcurr, success, iters, total))
    done = done + curr+", "

    if success:
      print(f"    ✅ SUCCESS after {iters} iteration(s) ({total:.1f}s)")
    else:
      print(f"    ❌ FAILED after {iters} iteration(s) ({total:.1f}s)")

  # Print summary
  print("\n" + "="*70)
  print("GENERATION SUMMARY")
  print("="*70)

  passed = sum(1 for r in results if r[1])
  failed = len(results) - passed

  print(f"\n{'Module':<30} {'Status':<10} {'Iters':<8} {'Time':<10}")
  print("-"*60)
  for name, success, iters, time_taken in results:
    status = "✅ PASS" if success else "❌ FAIL"
    print(f"{name:<30} {status:<10} {iters:<8} {time_taken:.1f}s")

  print("-"*60)
  print(f"\nResults: {passed} passed, {failed} failed out of {len(results)} modules")
  print(f"\nTotal time: {np.sum(totaltime):.1f}s")
  print(f"  - Generation: {np.sum(gentime):.1f}s")
  print(f"  - Verification: {np.sum(errortime):.1f}s")
  print("="*70)

  if failed > 0:
    print("\n⚠️  Some modules failed. Check the log files in each module's directory.")
  else:
    print("\n🎉 All modules generated successfully!")

# Setting the API Key

In [ ]:
### API KEY CONFIGURATION

# Option 1: Use Colab secrets (recommended)
from google.colab import userdata
#os.environ["OPENAI_API_KEY"] = userdata.get('ASIC-Deep-Dive')

# Option 2: Set directly (not recommended for shared notebooks)
#os.environ["OPENAI_API_KEY"] = "API KEY HERE"
#os.environ['CLAUDE_API_KEY'] = "API KEY HERE"
#os.environ['GEMINI_API_KEY'] = "API KEY HERE"



# Fetch API key ONCE and store in environment variable
os.environ["OPENAI_API_KEY"] = userdata.get("ROME-Colab")
# Select which model to use
os.environ["MODEL"] = "ChatGPT"
#os.environ["MODEL"] = "Claude"
#os.environ["MODEL"] = "Gemini"

# Accumulator Project Setup

In [ ]:
#@title Upload Testbenches
#
# ============================================================================
# STEP-BY-STEP INSTRUCTIONS FOR UPLOADING TESTBENCHES
# ============================================================================
#
# STEP 1: Download the testbenches
#   - Download accumulator_project.zip from the course materials
#   - Extract the zip file on your local computer
#   - Navigate to the extracted folder: accumulator_project/testbenches/
#
# STEP 2: Open the Colab file browser
#   - Look at the LEFT SIDE of this Colab window
#   - Click the FOLDER ICON to open the file browser panel
#
# STEP 3: Upload the testbench files
#   - At the TOP of the file browser panel, click the UPLOAD button
#   - Select ALL the .v files from the testbenches folder
#   - Click Open to upload them
#
# STEP 4: Verify the upload by running this cell
# ============================================================================

import os

required_testbenches = [
    "parity_gen_32bittb.v",
    "rectifier_12bittb.v",
    "divider_shifttb.v",
    "sample_countertb.v",
    "irq_controllertb.v",
    "accumulator_coretb.v",
    "sra_controllertb.v",
    "accum_toptb.v",
]

print("Checking for testbench files...")
print("=" * 50)

missing_files = []
found_files = []

for tb in required_testbenches:
    if os.path.exists(tb):
        print(f"  ✓ {tb}")
        found_files.append(tb)
    else:
        print(f"  ✗ {tb} - NOT FOUND")
        missing_files.append(tb)

print("=" * 50)
print(f"Found: {len(found_files)}/{len(required_testbenches)} testbenches")

if missing_files:
    print("\n⚠️  MISSING FILES - Please upload these before proceeding:")
    for f in missing_files:
        print(f"    - {f}")
else:
    print("\n✅ All testbenches are present! You can proceed to generation.")

Checking for testbench files...
  ✓ parity_gen_32bittb.v
  ✓ rectifier_12bittb.v
  ✓ divider_shifttb.v
  ✓ sample_countertb.v
  ✓ irq_controllertb.v
  ✓ accumulator_coretb.v
  ✓ sra_controllertb.v
  ✓ accum_toptb.v
Found: 8/8 testbenches

✅ All testbenches are present! You can proceed to generation.


# Tier 1: Basic Building Blocks

Parity generator, rectifier, and shift divider - no dependencies.

In [ ]:
#@title Tier 1 Submodules Definition

submodules_tier1 = [
    ["parity_gen_32bit",
     "32-bit even parity generator for register protection",
     "input wire [31:0] data_in, output wire parity_bit"],

    ["rectifier_12bit",
     "12-bit signed to unsigned rectifier (absolute value)",
     "input wire signed [11:0] data_in, input wire enable, output wire [11:0] data_out"],

    ["divider_shift",
     "32-bit right-shift divider with optional rounding, shift amount 0-11",
     "input wire signed [31:0] data_in, input wire [3:0] shift_amount, input wire round_en, output wire signed [31:0] data_out"],
]

In [ ]:
#@title Run Tier 1 Generation

os.environ["MODEL"] = "ChatGPT"
hier_gen(submodules_tier1, max_iterations=10)

HIERARCHICAL VERILOG GENERATION
Generating 3 modules with max 10 iterations each

[1/3] MODULE: parity_gen_32bit
    Description: 32-bit even parity generator for register protection
    Testbench: parity_gen_32bittb.v
--------------------------------------------------
    Attempt 1/10: Generating code... ✅ PASSED
    ✅ SUCCESS after 1 iteration(s) (1.3s)

[2/3] MODULE: rectifier_12bit
    Description: 12-bit signed to unsigned rectifier (absolute value)
    Testbench: rectifier_12bittb.v
--------------------------------------------------
    Attempt 1/10: Generating code... ❌ TEST FAILED
    Attempt 2/10: Generating code... ✅ PASSED
    ✅ SUCCESS after 2 iteration(s) (8.5s)

[3/3] MODULE: divider_shift
    Description: 32-bit right-shift divider with optional rounding, shift amount 0-11
    Testbench: divider_shifttb.v
--------------------------------------------------
    Attempt 1/10: Generating code... ❌ COMPILE ERROR
    Attempt 2/10: Generating code... ❌ COMPILE ERROR
    Attempt

# Tier 2: Counters and Control

Sample counter and IRQ controller.

In [ ]:
#@title Tier 2 Submodules Definition

submodules_tier2 = [
    ["sample_counter",
     "12-bit down counter: loads load_value when load=1, decrements on each sample_valid pulse when enable=1 and count>0, outputs done=1 for one cycle when count transitions from 1 to 0",
     "input wire clk, input wire rst_n, input wire [11:0] load_value, input wire load, input wire enable, input wire sample_valid, output reg [11:0] count, output reg done"],

    ["irq_controller",
     "Single-bit IRQ latch: on posedge clk, if irq_set=1 then irq_pending<=1, if irq_clear=1 then irq_pending<=0, irq_out is combinational irq_pending AND irq_en, async reset clears irq_pending to 0",
     "input wire clk, input wire rst_n, input wire irq_set, input wire irq_en, input wire irq_clear, output reg irq_pending, output wire irq_out"],
]

In [ ]:
#@title Run Tier 2 Generation

os.environ["MODEL"] = "ChatGPT"
hier_gen(submodules_tier2, max_iterations=10)

HIERARCHICAL VERILOG GENERATION
Generating 2 modules with max 10 iterations each

[1/2] MODULE: sample_counter
    Description: 12-bit down counter: loads load_value when load=1, decrements on each sample_valid pulse when enable=1 and count>0, outputs done=1 for one cycle when count transitions from 1 to 0
    Testbench: sample_countertb.v
--------------------------------------------------
    Attempt 1/10: Generating code... ✅ PASSED
    ✅ SUCCESS after 1 iteration(s) (2.9s)

[2/2] MODULE: irq_controller
    Description: Single-bit IRQ latch: on posedge clk, if irq_set=1 then irq_pending<=1, if irq_clear=1 then irq_pending<=0, irq_out is combinational irq_pending AND irq_en, async reset clears irq_pending to 0
    Testbench: irq_controllertb.v
--------------------------------------------------
    Attempt 1/10: Generating code... ❌ COMPILE ERROR
    Attempt 2/10: Generating code... ✅ PASSED
    ✅ SUCCESS after 2 iteration(s) (6.1s)

GENERATION SUMMARY

Module                         S

# Tier 3: Core Accumulator

32-bit signed accumulator core.

In [ ]:
#@title Tier 3 Submodules Definition

submodules_tier3 = [
    ["accumulator_core",
     "32-bit signed accumulator using 2's complement arithmetic: on posedge clk, if clear=1 then sum<=0, else if sample_valid=1 then sum<=sum+sample_in (sign-extended), async reset sets sum to 0",
     "input wire clk, input wire rst_n, input wire signed [11:0] sample_in, input wire sample_valid, input wire clear, output reg signed [31:0] sum"],
]


In [ ]:
#@title Run Tier 3 Generation

os.environ["MODEL"] = "ChatGPT"
hier_gen(submodules_tier3, max_iterations=10)

HIERARCHICAL VERILOG GENERATION
Generating 1 modules with max 10 iterations each

[1/1] MODULE: accumulator_core
    Description: 32-bit signed accumulator using 2's complement arithmetic: on posedge clk, if clear=1 then sum<=0, else if sample_valid=1 then sum<=sum+sample_in (sign-extended), async reset sets sum to 0
    Testbench: accumulator_coretb.v
--------------------------------------------------
    Attempt 1/10: Generating code... ❌ TEST FAILED
    Attempt 2/10: Generating code... ✅ PASSED
    ✅ SUCCESS after 2 iteration(s) (6.8s)

GENERATION SUMMARY

Module                         Status     Iters    Time      
------------------------------------------------------------
accumulator_core               ✅ PASS     2        6.8s
------------------------------------------------------------

Results: 1 passed, 0 failed out of 1 modules

Total time: 6.8s
  - Generation: 6.8s
  - Verification: 0.0s

🎉 All modules generated successfully!


# Tier 4: Top Level

SRA controller and complete accumulator top module.

In [ ]:
#@title Tier 4 Submodules Definition

submodules_tier4 = [
    ["sra_controller",
     "SRA state machine: when enable=1, first counts warmup_delay samples (in_warmup=1), then counts sra_length samples (accumulating=1), pulses sra_done=1 for one cycle when length reached, then repeats accumulation (no warmup on subsequent periods). length_count counts up from 0 to sra_length. If sra_length=0, sra_done never pulses.",
     "input wire clk, input wire rst_n, input wire enable, input wire sample_valid, input wire [7:0] warmup_delay, input wire [11:0] sra_length, output reg in_warmup, output reg accumulating, output reg sra_done, output reg [7:0] warmup_count, output reg [11:0] length_count"],

    ["accum_top",
     "Top-level accumulator: sample_valid_int = |(signal_bus_valid & reg_source); sample_in = signal_bus_data[11:0]; enable=reg_cfg[0]; abs_en=reg_cfg[1]; round_en=reg_cfg[2]; irq_en=reg_cfg[3]. When sample_valid_int & enable: if abs_en, rectify sample_in, then add to 32-bit sum. reg_sum outputs sum. On reg_clear, clear sum. For SRA: use reg_length samples, divide sum by 2^reg_divide_factor with round_en, store in reg_sra_result, auto-clear sum. Pulse IRQ on SRA done if irq_en. reg_warmup_delay delays first period. parity_error always 0 for now.",
     "input wire clk, input wire rst_n, input wire [31:0] signal_bus_data, input wire [31:0] signal_bus_valid, input wire [31:0] reg_source, input wire [3:0] reg_cfg, input wire reg_clear, input wire [11:0] reg_length, input wire [3:0] reg_divide_factor, input wire [7:0] reg_warmup_delay, input wire reg_irq_clear, output reg [31:0] reg_sum, output reg [31:0] reg_sra_result, output reg [7:0] reg_warmup_delay_count, output reg [11:0] reg_sra_length_count, output reg reg_irq, output wire irq_out, output wire parity_error"],
]


In [ ]:
#@title Run Tier 4 Generation

os.environ["MODEL"] = "ChatGPT"
hier_gen(submodules_tier4, max_iterations=10)

HIERARCHICAL VERILOG GENERATION
Generating 2 modules with max 10 iterations each

[1/2] MODULE: sra_controller
    Description: SRA state machine: when enable=1, first counts warmup_delay samples (in_warmup=1), then counts sra_length samples (accumulating=1), pulses sra_done=1 for one cycle when length reached, then repeats accumulation (no warmup on subsequent periods). length_count counts up from 0 to sra_length. If sra_length=0, sra_done never pulses.
    Testbench: sra_controllertb.v
--------------------------------------------------
    Attempt 1/10: Generating code... ✅ PASSED
    ✅ SUCCESS after 1 iteration(s) (8.5s)

[2/2] MODULE: accum_top
    Description: Top-level accumulator: sample_valid_int = |(signal_bus_valid & reg_source); sample_in = signal_bus_data[11:0]; enable=reg_cfg[0]; abs_en=reg_cfg[1]; round_en=reg_cfg[2]; irq_en=reg_cfg[3]. When sample_valid_int & enable: if abs_en, rectify sample_in, then add to 32-bit sum. reg_sum outputs sum. On reg_clear, clear sum. For S

# Complete Module List

All 8 modules for reference.

In [ ]:
#@title All Submodules (for reference)

# Complete list of all 8 modules
submodules_all = [
    # Tier 1
    ["parity_gen_32bit", "32-bit even parity generator", "input wire [31:0] data_in, output wire parity_bit"],
    ["rectifier_12bit", "12-bit signed to unsigned rectifier", "input wire signed [11:0] data_in, input wire enable, output wire [11:0] data_out"],
    ["divider_shift", "32-bit right-shift divider with optional rounding", "input wire signed [31:0] data_in, input wire [3:0] shift_amount, input wire round_en, output wire signed [31:0] data_out"],
    # Tier 2
    ["sample_counter", "Sample counter with done pulse", "input wire clk, input wire rst_n, input wire [11:0] load_value, input wire load, input wire enable, input wire sample_valid, output wire [11:0] count, output wire done"],
    ["irq_controller", "IRQ with enable and W1C", "input wire clk, input wire rst_n, input wire irq_set, input wire irq_en, input wire irq_clear, output wire irq_pending, output wire irq_out"],
    # Tier 3
    ["accumulator_core", "32-bit signed accumulator", "input wire clk, input wire rst_n, input wire signed [11:0] sample_in, input wire sample_valid, input wire clear, output reg signed [31:0] sum"],
    # Tier 4
    ["sra_controller", "SRA state machine", "input wire clk, input wire rst_n, input wire enable, input wire sample_valid, input wire [7:0] warmup_delay, input wire [11:0] sra_length, output wire in_warmup, output wire accumulating, output wire sra_done, output wire [7:0] warmup_count, output wire [11:0] length_count"],
    ["accum_top", "Top-level accumulator", "input wire clk, input wire rst_n, input wire [31:0] signal_bus_data, input wire [31:0] signal_bus_valid, input wire [31:0] reg_source, input wire [3:0] reg_cfg, input wire reg_clear, input wire [11:0] reg_length, input wire [3:0] reg_divide_factor, input wire [7:0] reg_warmup_delay, input wire reg_irq_clear, output wire [31:0] reg_sum, output wire [31:0] reg_sra_result, output wire [7:0] reg_warmup_delay_count, output wire [11:0] reg_sra_length_count, output wire reg_irq, output wire irq_out, output wire parity_error"],
]

print(f"Total modules: {len(submodules_all)}")

In [ ]:
#@title Run All Modules Generation

# Use this to generate all modules at once
# os.environ["MODEL"] = "ChatGPT"
# hier_gen(submodules_all, max_iterations=10)
print("Uncomment and run to generate all 8 modules")

# Complete Module List

All 8 modules for reference.